In [1]:
from serpapi import GoogleSearch
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

In [2]:
def get_newspapers(query):
  params = {
    "api_key": "d8b42ca90077ec59802d53ad3d55e7e05d38ebac54426fbfa8913b8f40f68e24",
    "engine": "google_news",
    "hl": "en",
    "gl": "us",
    "q": query
  }

  search = GoogleSearch(params)
  results = search.get_dict()
  article_metadata = results['news_results']
  newspapers = []

  for article in tqdm(article_metadata):
      article_link = article.get('link', '')
      article_title = article.get('title', 'No title')
      article_publication_date = article.get('date', 'No date')
      try:
          time.sleep(5)
          response = requests.get(article_link, timeout=30)
          response.raise_for_status()
          soup = BeautifulSoup(response.content, 'html.parser')
          article_text = ""
          for paragraph in soup.find_all('p'):
              article_text += paragraph.get_text() + "\n"

          items = {
              'title': article_title,
              'date': article_publication_date,
              'link': article_link,
              'text': article_text,
          }
          newspapers.append(items)


      except requests.exceptions.RequestException as e:
          pass
      except Exception as e:
          pass
  return pd.DataFrame(newspapers)

In [3]:
newspapers = get_newspapers("donald trump")
newspapers.to_csv('trump.csv')

100%|██████████| 100/100 [09:00<00:00,  5.40s/it]


In [7]:
newspapers.shape

(58, 4)